In [110]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

Improt the dataset and clean it before training

In [111]:
#setup display options
pd.set_option('display.max_columns', None)
pd.set_option('display.min_rows', 15)

df = pd.read_csv("..\data\data_cleaned.csv",index_col=[0])


#replace categorical data with number
df["kitchen"].fillna("Not equipped",inplace=True)

#get dummies for catagorical data:
get_dummies = ["kitchen","type of property","postalCode","state of the building"]
for prop in get_dummies:
    dummy_data=pd.get_dummies(df[prop]).astype(int)
    df = df.join(dummy_data)

df.drop(columns=["fully equipped kitchen"],inplace=True)

#replace True with 1, and False with 0, turn them into numerical data
df.replace([True,False],[1,0],inplace=True)



In [112]:
df.shape

(15670, 946)

Remove outliners


In [113]:
def remove_outliners(df,columns,n_std):
    for col in columns:

        mean = df[col].mean()
        sd = df[col].std()
        df = df[(df[col] <= mean+(n_std*sd))]
        df = df[(df[col] >= mean-(n_std*sd))]
    return df

# def remove_outliner(df, col_names):
#     for col_name in col_names:
#         q1 = df[col_name].quantile(0.25)
#         q3 = df[col_name].quantile(0.75)
#         iqr = q3-q1 #Interquartile range
#         fence_low  = q1-1.5*iqr
#         fence_high = q3+1.5*iqr
#         df = df.loc[(df[col_name] > fence_low) & (df[col_name] < fence_high)]

#     return df



In [114]:
df = remove_outliners(df,["number of bedrooms","total property area","livable area", "garden area"],3)
df.shape

(15020, 946)

Data formatting: create X,y, split them into training and testing

In [115]:
# fix the distributions of number of bedrooms
df["number of bedrooms"] = np.log(df["number of bedrooms"] + 1)

In [116]:
# price would be the target
columns_to_drop = ["id","type of property","subtype of property","locality","latitude","longitude","street","price","type of sale",
                   "state of the building","Gemeente","Province","Density","Area(per province)",
                   "Population( per province)","kitchen","price per sqr"]
X_raw = df.drop(columns=columns_to_drop,axis=1).to_numpy()
y_raw = df.loc[:,"price"].to_numpy().reshape(-1,1)

#Now normalize the data
# X=preprocessing.normalize(X_raw) 
# print(X.shape,y.shape)
X_train, X_test, y_train, y_test = train_test_split(X_raw, y_raw, train_size= 0.20)


In [117]:
print(X_raw.shape, X_train.shape, y_train.shape)

(15020, 929) (3004, 929) (3004, 1)


visualization of the raw_data

In [118]:

# df[columns_to_select].hist(figsize=(15,8))



Model selection

In [119]:
reg = LinearRegression()
reg.fit(X_train, y_train)

# y_pred = reg.predict(X_test)
print(reg.score(X_train,y_train))


0.6759181430958178
